In [29]:
import seaborn as sns
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Give the path of your dataset below and run

In [41]:
# paste your file path in the space given below
dataframe = pd.read_csv('#Give File Path Here')

In [42]:
# This is not to change anything in your original data Frame
df = dataframe.copy()

#### Give the target variable name below and run

In [43]:
# Give your output label here
df=df.rename(columns = {'#ENTER HERE':'output'})

### From here run all the cells to see how your data is ditributed no need to change anything Just run the cells

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.shape

### Just run the cells below to visualize your data 

### Visualizing the data

In [ ]:
# Heatmap for null values
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
#Box-Plot to understand the distribution of numerical data.
Numerical_Features = [feature for feature in df.columns if df[feature].dtypes!='O']
for feature in Numerical_Features:
    df.boxplot(column=feature)
    plt.ylabel(feature)
    plt.title(feature)
    plt.show()

In [ ]:
# Scatter plots to show the relation between your Numerical features and outputlable
for feature in Numerical_Features:
    if feature != 'output':
        #df[feature]=np.log(df[feature])
        #df['output']=np.log(df['output'])
        plt.scatter(df[feature],df['output'])
        plt.xlabel(feature)
        plt.ylabel('output')
        plt.title(feature)
        plt.show()


In [ ]:
# Bar plots to show the relation between your catogorical(text) features and outputlable
Catogorical_features = [feature for feature in df.columns if df[feature].dtypes=='O'] 
for feature in Catogorical_features:
    if feature != 'output':
        df.groupby(feature)['output'].median().plot.bar()
        plt.xlabel(feature)
        plt.ylabel('output')
        plt.title(feature)
        plt.show()

##### Correlation Heatmap,On analysing this you can drop the features that are not useful like if some more correlated drop them otherwise they will act like duplicates

In [ ]:
df.drop(columns='output',axis=1)
df.corr()

In [ ]:
#Using Pearson Correlation
dp = df.copy() #copying so that we dont change anything in our original dataframe 
dp = dp.drop('output',axis = 1)
plt.figure(figsize=(12,10))
cor = dp.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.CMRmap_r)
plt.show()

In [21]:
# This function will give you the correlated features based on the threshold you set
def correlation(data, threshold):
    col_corr = set() 
    corr_matrix = data.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: #Taking abs value because (pearson corr) have corr range of (-1,1).so we need only +highvalues.
                colname = corr_matrix.columns[i] 
                col_corr.add(colname)
    return col_corr

In [22]:
corr_features = correlation(df, 0.8)
print("No. of correlated features are {}".format(len(set(corr_features))))

No. of correlated features are 4


In [ ]:
# you can drop these corr_features both from your train data and test data to avoid overfitting.
#df.drop(corr_features,axis = 1) #use this if you want to drop

#### Preprocessing Text Features(These are just basic preprocessing You can do more advance,but this will help in solving)

> Convert your text features into numericals



In [46]:
# Guys if you have less(say < 30) no. of catogories in a feature use this other wise apply one-hoting to the top 10 or 20 catogories in your data
df = pd.concat([df,pd.get_dummies(df[Catogorical_features],drop_first=True)], axis=1)
df.drop(Catogorical_features,axis=1,inplace=True)

#### Test-Train Split

In [25]:
Y = df.output
X = df.drop(columns = 'output',axis = 1)

In [48]:
# Like below do your validate split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.20,random_state=42)

### Models

> Classification(If your problem is classification use these algorithms and choose the best for your problem statement)

1.   KNN
2.   DecisionTree
3.   SVM
4.   Logistic

## KNN

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [ ]:
error_rate = []
# It Will take some time
for i in range(1,40):# We are checking with several k values to know the best k
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train,y_train)
    pred_i = knn.predict(x_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
#From this graph choose best k(where error is less) and train with it
plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [ ]:
# Type your best k value
knn = KNeighborsClassifier(n_neighbors=#enter here)

knn.fit(x_train,y_train)
pred = knn.predict(x_test)

print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))

## DecisionTree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
TREE2 = DecisionTreeClassifier(criterion="entropy", max_depth = 6)
TREE2.fit(x_train,y_train)

In [ ]:
predDT = TREE2.predict(x_test)
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, predDT))

In [ ]:
from sklearn.metrics import classification_report,jaccard_similarity_score,log_loss,f1_score
print(classification_report(y_test,predDT))
print('\n')
print('Jaccard Similarity Score : ',round(jaccard_similarity_score(y_test,predDT)*100,2),'%')
print('\n')
print('F1-SCORE : ',f1_score(y_test,predDT,average=None))
print('\n')
print('Train Accuracy: ',metrics.accuracy_score(y_train, TREE2.predict(x_train))*100,'%')

## SupportVectorMachine(SVM)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
svm = SVC(kernel='rbf').fit(x_train,y_train)
pred_svm=svm.predict(x_test)

In [ ]:
print(classification_report(y_test,pred_svm))
print('\n')
print('Jaccard Similarity Score : ',round(jaccard_similarity_score(y_test,pred_svm)*100,2),'%')
print('\n')
print('F1-SCORE : ',f1_score(y_test,pred_svm,average=None))
print('\n')
print('Train Accuracy: ',metrics.accuracy_score(y_train, svm.predict(x_train))*100,'%')

In [ ]:
print('\n')
print('Test Accuracy: ',metrics.accuracy_score(pred_svm, y_test)*100,'%')
print('\n')

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lgm=LogisticRegression()
lgm.fit(x_train,y_train)
pred_lgm=lgm.predict(x_test)

In [ ]:
print(classification_report(y_test,pred_lgm))
print('\n')
print('Jaccard Similarity Score : ',round(jaccard_similarity_score(y_test,pred_lgm)*100,2),'%')
print('\n')
print('F1-SCORE : ',f1_score(y_test,pred_lgm,average=None))
print('\n')
print('Train Accuracy: ',metrics.accuracy_score(y_train, lgm.predict(x_train))*100,'%')

In [ ]:
print('\n')
print('Test Accuracy: ',metrics.accuracy_score(y_test, pred_lgm)*100,'%' )
print('\n')